In [1]:
%run stdPackages.ipynb

# Marginal Abatement Costs - BYOC

In this exercise, we'll build our own classes to solve some of the same problems as we did in problem set 1. The notebook only contains snippet of code; it is, generally, better practice to write classes and more involved code in designated ```.py``` files. In this example, you should add it to the file ```/pyfiles/E2.py```.

*This snippet loads the classes/methods that you've defined in ```pyfiles/E2.py:```*

In [2]:
os.chdir(d['curr'])
from pyfiles.E2 import MAC, MACTech

## Q1. A Toy Model of Abatement Costs

### **Q 1.1. Define base class**

*Define a class ```MAC``` with a constructor method (```__init__```) that:*
1. *Adds a database attribute as ```self.db``` as a dictionary.*
2. *Adds values for all relevant parameters to the database.*
3. *Allow you to modify default values by passing them to the ```__init__``` method.*
4. *Add methods that allows you to compute $C(E)$, $M(E)$, $\tilde{C}(E), MAC(E) = (F'(E)-p_e)/\phi$ all on grids of $E$.*

*You should be able to get the value of a symbol x by calling ```self.db['x']```*

### **Q 1.2. Add baseline and optimal solutions as methods to the class**

*Next, define methods that allow you to compute a baseline solution and an optimal one.*

### **Q 1.3. Initialize instance of the class and plot $E^*$ as a function of $\gamma$ and $\phi$.**

*Hint: It should end up looking something like this*

In [4]:
IFrame("./Figs/S2_Q13.pdf", width=800, height=400)

## Q2. Technical abatement as a child-class

In this exercise, we'll build a class on top of the parent class from **Q1** to look a bit into inheritance. As you'll see later in the course, (almost) all of the energy system models we build are children classes of the base class ```modelShell``` from ```lpModels```.

*NB: Some of the methods in ```MAC``` changes when we go to ```MACTech``` (e.g. the way to compute $C$ and $M$). As it is best practice to make sure that the parent class (```MAC```) methods doesn't break with child classes, we've added the argument ```**kwargs``` a couple of places in the base class. This allows us to add new arguments in ```MACTech``` methods that would simply not impact the base methods.*

### **Q 2.1. Briefly explain the initial part of this class**

```python
class MACTech(MAC):
    def __init__(self, α = .5, γ = 1, pe = 1, ϕ = .25, γd = 100, θ = None, c = None, σ = None):
        super().__init__(α = α, γ = γ, pe = pe, ϕ = ϕ, γd = γd) # use __init__ method from parent class
        self.initTechs(θ = θ, c = c, σ = σ)
        
    def initTechs(self, θ = None, c = None, σ = None):
        """ Initialize technologies from default values """
        if θ is None:
            self.db['Tech'] = pd.Index(['T1'], name = 'i')
            self.db['θ'] = pd.Series(0, index = self.db['Tech'], name = 'θ')
            self.db['c'] = pd.Series(1, index = self.db['Tech'], name = 'c')
            self.db['σ'] = pd.Series(1, index = self.db['Tech'], name = 'σ')
        elif isinstance(θ, pd.Series):
            self.db['θ'] = θ
            self.db['c'] = c
            self.db['σ'] = σ
            self.db['Tech'] = self.db['θ'].index
        else:
            self.db['Tech'] = 'T'+pd.Index(range(1, len(θ)+1), name = 'i').astype(str)
            self.db['θ'] = pd.Series(θ, index = self.db['Tech'], name = 'θ')
            self.db['c'] = pd.Series(c, index = self.db['Tech'], name = 'c')
            self.db['σ'] = pd.Series(1, index = self.db['Tech'], name = 'σ')
```

### **Q 2.2. Add relevant methods to the class**

1. *Add methods that determine optimal abatement and abatement costs given marginal damages.*
2. *Adjust existing methods for computing $C$, $\tilde{C}$ to this class with technical abatement (now they are functions of $M$ as well)*
3. *Add/adjust the methods that define the optimal solution for $E,M,C$. using the methods in **Q2.4** in PS1.*

### **Q 2.3. Compute marginal abatement costs on a grid of abated emissions**

*Start with an initialization of your class:*

In [5]:
Tech = pd.Index(['T1','T2', 'T3'], name = 'i')
θ = pd.Series([0.25, 0.25, 0.25], index = Tech, name  = 'θ')
c = pd.Series([0.5, 2, 5], index = Tech, name = 'c')
σ = pd.Series([.025, .025, .025], index = Tech, name = 'σ')
mTech = MACTech(θ = θ, c= c, σ=σ)

*Finally, we want a method that allows us to replicate figure 2.2. in the lecture note with slightly different technology parameters. To do this, we need to go through a couple of steps. We start from equation (8) that defines marginal abatement costs. This equation currently depends on $E$ and technical abatement variables $a_i$ that ultimately depends on the marginal damages in optimum. To get at the MAC curve, proceed as follows:*

1. *Form a grid of marginal damages ```mdGrid``` as a linear space from 0 to 10.*
2. *For each $md_j \in$ ```mdGrid```, compute the right-hand side of eq. (7) that defines optimality given marginal damages. Define this vector ```mcGrid```.*
3. *Given the value of $mc_j\in$ ```mcGrid```, we can now back out the corresponding level of $E_j$ from (7), that is:*
$$\begin{align}
    \dfrac{F'(E_j)-p_e}{\phi} = mc_j \qquad \Rightarrow \qquad E_j = \left(\dfrac{\alpha \gamma}{\phi mc_j+p_e}\right)^{\frac{1}{1-\alpha}}
\end{align}$$
    *We define the vector of these $E_j$ as ```Egrid```*
4. *Next, for each $md_j\in$ ```mdGrid```, compute emissions from $M = E \phi (1-\sum_i\theta_ia_i)$ and define this vector ```Mgrid```.*
5. *Finally, exploit that $MAC = D'(M)$, to define MAC on the grid of abated emissions. (this is a pandas series with values = ```mdGrid``` and index = M0-```Mgrid```)*
6. *Plot this against the MAC-curve without technical effects. Briefly comment.*

In [6]:
mdGrid = np.linspace(np.finfo(float).eps, 10, 1000)

*Hint: With the chosen parameter values, it'll end up looking something like this*

In [ ]:
IFrame("./Figs/S2_Q23.pdf", width=800, height=400)